# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


- Nama: Rayhan Egar Sadtya Nugraha
- NIM: 225150201111014
- Kelas: PPM-D

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu




Cek isi dataset Anda dengan menggunakan perintah **head()**

Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

Tampilkan beberapa data untuk mengecek hasilnya

## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?